In [ ]:
from qampy import signals, impairments, equalisation, phaserec, helpers
import numpy as np
# here we use bokeh for plotting as it is much faster
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
output_notebook()



In [ ]:
# setup some data first because we want to demo the time of the equaliser
sigs = []
for i in range(5):
    s = signals.SignalQAMGrayCoded(4, 10**5, nmodes=2, fb=40e9)
    s = s.resample(2*s.fb, beta=0.1, renormalise=True)
    s = impairments.change_snr(s, 13)
    s = impairments.apply_PMD(s, np.pi/5.5, 50e-12)
    sigs.append(s)
fig = figure(title="QPSK signal constellation", output_backend="webgl")
Xp = fig.scatter(x=s[0].real, y=s[0].imag,  color='blue', alpha=0.3)
Yp = fig.scatter(x=s[1].real, y=s[1].imag,  color='red', alpha=0.3)


## QAMpy and Open Source DSP chain

## QAMPy running equalisation on 100 000 symbol PM-QPSK

In [ ]:
handle=show(fig, notebook_handle=True)
for i in range(50):
    wxy, err = equalisation.equalise_signal(sigs[i%5], 2e-3, Ntaps=17, method="mcma", adaptive_stepsize=True)
    sout = equalisation.apply_filter(sigs[i%5], wxy)
    sout = helpers.normalise_and_center(sout)
    # note plotting is a significant bottleneck if we plot all points!
    Xp.data_source.data["x"] = sout[0][::10].real
    Xp.data_source.data["y"] = sout[0][::10].imag
    Yp.data_source.data["x"] = sout[1][::10].real
    Yp.data_source.data["y"] = sout[1][::10].imag
    push_notebook()